# Pymongo import

In [69]:
import pymongo
from pymongo import MongoClient
import datetime

DB 커넥션 만들기

In [4]:
def get_db_connection(tbl_name):
    # connect to database
    connection = MongoClient('localhost',27017)

    db = connection.swordfish

    if (tbl_name == "livetalk"):
        cur_tbl = db.livetalk
    elif (tbl_name == "estimate"):
        cur_tbl = db.estimate
    elif (tbl_name == "sr"):
        cur_tbl = db.sr

    return cur_tbl

In [135]:
tbl = get_db_connection('livetalk')

샘플 데이터 mongodb에 넣기


In [71]:
tbl.insert_one(dic)

# ItemCode로 Query하기

In [62]:
cursor = tbl.find({'item_code':477816})

# Query datas with more params(item_code,startDate,endDate)

function to convert cursor to list type

In [129]:
import json
from bson.json_util import dumps,loads

def convertCursorToString(cursor):
    temp = dumps(cursor)
    
    return temp.encode('euc-kr')

In [125]:
def convertCursorToList(cursor):
    temp = dumps(cursor)

    resultList = loads(temp)
    
    return resultList

Extract whole contents list data with three params
- item_code
- startDate
- endDate

In [140]:
def getCntntListByIdAndDate(item_code,startDate,endDate):
    cursor = tbl.find({'item_code':int(item_code),"reg_dt":{"$gte":startDate, "$lte":endDate}},{'cntnt_list':1,'_id':0})

    return convertCursorToString(cursor)

Extract nouns list with three params
- item_code
- startDate
- enddate

In [141]:
def getNounsListByIdAndDate(item_code,startDate,endDate):
    cursor = tbl.find({'item_code':int(item_code),"reg_dt":{"$gte":startDate, "$lte":endDate}},{'cntnt_list.nouns_list':1,'_id':0})

    return convertCursorToString(cursor)

Extract item_code and item_name between two datetime

In [113]:
def getItemCodeAndNameByTblName(tblName,startDate,endDate):
    tbl = get_db_connection(tblName)
    
    cursor = tbl.find({"reg_dt":{"$gte":startDate, "$lte":endDate}},{'item_code':1,'item_name':1,'_id':0})
    
    return cursor

tempCur = getItemCodeAndNameByTblName('livetalk',convertStringToDatetime('20160601'),convertStringToDatetime('20160630'))
for row in tempCur:
    print(row)

{'item_name': '한국블루베리영농조합:기타신선과일', 'item_code': 477816}
{'item_name': '데시구알:샌들/슬리퍼', 'item_code': 307605}


Convert String to Datetime type

In [115]:
from datetime import datetime

def convertStringToDatetime(str_datetime):

    y = datetime.strptime(str_datetime, '%Y%m%d')
    
    return y

In [139]:
returnCursor = getNounsListByIdAndDate(477816,convertStringToDatetime('20160601'),convertStringToDatetime('20160630'))

returnCursor

[{'cntnt_list': [{'nouns_list': ['애']},
   {'nouns_list': ['타임지', '선정', '푸드', '주문']},
   {'nouns_list': ['무', '농약', '구매']},
   {'nouns_list': ['미리']},
   {'nouns_list': ['부모님', '눈', '건강']},
   {'nouns_list': ['구입']},
   {'nouns_list': ['요거트', '스무디', '해']},
   {'nouns_list': ['실', '구매']},
   {'nouns_list': ['무', '농약', '시원']},
   {'nouns_list': ['브릭스']},
   {'nouns_list': ['왜', '아직', '방송', '가타', '방송', '구입', '샵', '믿음', '또', '구입']},
   {'nouns_list': ['블루베리', '차이']},
   {'nouns_list': ['법', '구매']},
   {'nouns_list': ['맛', '생', '과로']},
   {'nouns_list': ['얼른', '얼음', '슬러시']},
   {'nouns_list': ['아가', '워낙', '구매']},
   {'nouns_list': ['블루베리', '생', '가격']},
   {'nouns_list': ['냉장', '보관', '언제', '바로', '냉동']}]}]

# Pymongo cursor를 Json파일로 변환하기

In [121]:
import json

from bson.json_util import dumps

myjson = dumps(returnCursor,ensure_ascii=False)

myjson

'"[]"'

# Json 로딩

In [122]:
import urllib.request

url = "http://localhost:8000/livetalk/contents/477816/20160601/20160630/"
wp = urllib.request.urlopen(url)
response = wp.read().decode('euc-kr')

In [123]:
result = json.loads(response)

result

[]

In [50]:
import pandas as pd

r = pd.DataFrame(result)

r['cntnt_list'][0][0]['cntnt']

'애들이 좋아해요'